<a href="https://colab.research.google.com/github/bala-codes/Sentiment_Analysis_on_Amazon_Product_Reviews_Using_Machine_and_Deep_Learning/blob/master/codes%20(DL%20-%20RNN%20%26%20LSTM)/Part-2.1%20Sentiment%20Analysis%20-%20Training%20-%20Long%20Short%20Term%20Memory%20(LSTM)%20With%20Stanford's%20Glove%20Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Importing the libraries which are required.
import pandas as pd
import string
import pickle
import warnings
import numpy as np
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
try:
    from tensorflow.python.util import module_wrapper as deprecation
except ImportError:
    from tensorflow.python.util import deprecation_wrapper as deprecation
deprecation._PER_MODULE_WARNING_LIMIT = 0

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [ ]:
final_df = pd.read_csv('/content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/Amazon EWS - Final Dataset/finaldf_forlstm.csv') 
final_df = pd.DataFrame(final_df)

final_df.dropna(inplace=True)
final_df.head()

,Unnamed: 0,review_body_cleaned,star_rating_enc
0,0,GOOD VALUE,1.0
1,1,Excellent quality,1.0
2,2,season using it only fill with water and imme...,1.0
3,3,Nice bag Should another strap bottom somethin...,1.0
4,4,Gave my son used we backpacking loves it,1.0


In [ ]:
final_df['review_body_cleaned'] = final_df['review_body_cleaned'].astype(str)

In [ ]:
from sklearn.model_selection import train_test_split

sentences = final_df['review_body_cleaned'].values
y = final_df['star_rating_enc'].values

sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.20, random_state=123, shuffle=True)
print(sentences_train.shape, sentences_test.shape)
print(y_train.shape, y_test.shape)

(318595,) (79649,)
(318595,) (79649,)


In [ ]:
ytest1 = list(y_test)
print("1's",ytest1.count(1))
print("0's",ytest1.count(0))
print("Thus Test dataset is well Balanced")

1's 39697
0's 39952
Thus Test dataset is well Balanced


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1
print("Total Vocabulary Size", vocab_size)

Using TensorFlow backend.


Total Vocabulary Size 70763


In [ ]:
tokenizer.word_index.items()
with open('/content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/PACKAGE 3 -SOURCE CODES AND FILES/Pretrained Models/Deep Learning Models - With Glove Embedding/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 300

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    print("X",x)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

# Stanford's Glove Representational Embedding Layer Insertion

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-12 06:14:40--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-12 06:14:41--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-12 06:14:41--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
!unzip '/content/glove.6B.zip' -d '/content/Glove_Embedding/'


Archive:  /content/glove.6B.zip
  inflating: /content/Glove_Embedding/glove.6B.50d.txt  
  inflating: /content/Glove_Embedding/glove.6B.100d.txt  
  inflating: /content/Glove_Embedding/glove.6B.200d.txt  
  inflating: /content/Glove_Embedding/glove.6B.300d.txt  


#GLOVE HERE

In [ ]:
# load the whole embedding into memory

embeddings_index = dict()
embedding_dims = []

#f = open('/content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/PACKAGE 3 -SOURCE CODES AND FILES/Pretrained Models/Deep Learning Models - With Glove Embedding/glove.6B.100d.txt')

embedding_dims = input(print("Enter the Embedding Dimensions 50 or 100 or 200 or 300 : "))

f = open('/content/Glove_Embedding/glove.6B.{}d.txt'.format(embedding_dims))
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Loaded %s word vectors.' % len(embeddings_index))

################################################################################

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector


Enter the Embedding Dimensions 50 or 100 or 200 or 300 : 
None100
Loaded 400000 word vectors.


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GRU, Dropout, LSTM, GlobalMaxPooling1D
from keras.callbacks import *

model = Sequential()
model.add(Embedding(vocab_size, output_dim = int(embedding_dims) , weights=[embedding_matrix], input_length=maxlen, trainable=False))
model.add(LSTM(units = 196, recurrent_dropout = 0.4, dropout=0.4))  # return_sequences = True
# model.add(GlobalMaxPooling1D())
model.add(Dense(32,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 300, 100)          7076300   
_________________________________________________________________
lstm_5 (LSTM)                (None, 196)               232848    
_________________________________________________________________
dense_7 (Dense)              (None, 32)                6304      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 7,315,485
Trainable params: 239,185
Non-trainable params: 7,076,300
_________________________________________________________________


In [ ]:
mc = ModelCheckpoint('/content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/PACKAGE 3 -SOURCE CODES AND FILES/Pretrained Models/Deep Learning Models - With Glove Embedding/SA_LSTM_GLOVE{}.h5'.format(embedding_dims), monitor='val_acc', verbose=1, save_best_only=True, mode='max')
es = EarlyStopping(monitor='val_loss', patience=4)
red_lr= ReduceLROnPlateau(monitor='val_loss',patience=2,verbose=2,factor=0.001,min_delta=0.01)

In [ ]:
%%time
history = model.fit(X_train, y_train, epochs=10 , verbose= 2, validation_data=(X_test, y_test), callbacks=[mc,es,red_lr])

Train on 318595 samples, validate on 79649 samples
Epoch 1/10
 - 6439s - loss: 0.6934 - acc: 0.5005 - val_loss: 0.6930 - val_acc: 0.5024

Epoch 00001: val_acc improved from -inf to 0.50243, saving model to /content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/PACKAGE 3 -SOURCE CODES AND FILES/Pretrained Models/Deep Learning Models - With Glove Embedding/SA_LSTM_GLOVE100.h5
Epoch 2/10


In [ ]:
%%time
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

plot_history(history)

In [ ]:
model.save('/content/drive/My Drive/Machine Learning Projects/SENTIMENT ANALYSIS - AMAZON CUSTOMER REVIEWS/SOURCE CODES AND DATASETS/PACKAGE 3 -SOURCE CODES AND FILES/Pretrained Models/Deep Learning Models/sentiment_LSTM_GLOVE_Model.h5')
print("Model Saved Successfully")